In [153]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [159]:
with open("data/whitby_01_00_byu_Jordan_23.csv") as f:
    cols = f.readline()[1:-2]
cols = cols.split('","')
edge_cols = [c for c in cols if ("net@" in c and "choice" not in c)]
edge_cols.append("metric_id")

In [160]:
column_mapping = defaultdict(int, {c:str for c in edge_cols[:-1]})

In [161]:
df = pd.read_csv("data/whitby_01_00_byu_Jordan_23.csv", dtype=column_mapping, keep_default_na=False)
df = df.drop(df.loc[df["group_questionnaire_order"] == ""].index, axis=0)
df["group_questionnaire_order"] = df["group_questionnaire_order"].astype(float)
df


,group_level,group_language,group_school,group_code,group_type,group_id,group_questionnaire_order,group_late completion,metric_language_questionnaire,metric_test_date,...,net@37,net@38,net@39,net@40,net@41,net@1stchoice,net@2ndchoice,net@3rdchoice,net@4thchoice,net@5thchoice
0,3,ara,Qasid2022,,Semester,6,1.0,,eng,17407,...,100,100,130,210,x2x,30,14,8,24,10
1,3,ara,Qasid2022,,Semester,6,1.0,,eng,17407,...,252,xxx,354,358,xxx,39,6,37,28,19
2,3,ara,Qasid2022,,Semester,3,1.0,,eng,17407,...,226,118,xxx,123,112,,,,,
3,3,ara,Qasid2022,,Semester,4,1.0,,eng,17407,...,111,xxx,101,211,xxx,31,17,29,11,15
4,3,ara,Qasid2022,,Semester,1,1.0,,eng,17407,...,110,330,120,350,359,13,26,12,15,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,3,ara,Qasid2022,,Semester,2,3.0,,eng,17407,...,,,,,,,,,,
119,3,ara,Qasid2022,,Semester,2,3.0,,eng,17407,...,,,,,,,,,,
120,3,ara,Qasid2022,,Semester,6,3.0,,eng,17407,...,,,,,,,,,,
121,3,ara,Qasid2022,,Semester,3,3.0,,eng,17407,...,,,,,,,,,,


In [162]:
ddf = df.loc[df["group_questionnaire_order"] == 1.0]
ddf = ddf[edge_cols]
ddf


,net@1,net@2,net@3,net@4,net@5,net@6,net@7,net@8,net@9,net@10,...,net@33,net@34,net@35,net@36,net@37,net@38,net@39,net@40,net@41,metric_id
0,self,230,130,x?,?,120,130,150,110,240,...,100,120,230,100,100,100,130,210,x2x,1
1,145,self,130,240,220,143,110,100,110,120,...,242,xxx,244,210,252,xxx,354,358,xxx,2
2,118,201,self,205,238,224,147,142,118,224,...,247,127,126,133,226,118,xxx,123,112,3
3,xxx,xxx,xxx,self,201,101,101,xxx,xxx,113,...,211,137,xxx,xxx,111,xxx,101,211,xxx,4
4,121,120,151,120,self,110,100,110,210,130,...,210,230,240,150,110,330,120,350,359,5
5,120,153,131,x20,220,self,100,110,111,130,...,240,xxx,155,xxx,243,xxx,155,120,23x,6
6,230,220,210,200,200,200,self,120,200,130,...,246,210,200,200,246,146,201,250,13x,7
7,353,110,344,310,330,310,130,self,220,220,...,x10,x30,x10,x10,x10,xxx,xxx,104,200,8
8,120,210,230,120,220,110,110,130,self,153,...,120,231,110,245,220,251,113,110,x52,9
9,231,210,220,220,220,210,110,230,142,self,...,200,233,100,236,213,130,213,210,231,10


In [195]:
# def decode_edge(code: str, source: int, target: int) -> dict:
#     direction = None
#     contact_intensity = None
#     prct_language_used = None
#     try:
#         if code[0] == "1":
#             direction = "out"
#         elif code[0] == "2":
#             direction = "in"
#         elif code[0] == "3":
#             direction = "both"
#         contact_intensity = int(code[0]) / 5  # normalised
#         prct_language_used = int(code[2]) / 9  # normalised
#     except:
#         print(code)
#     return {
#         "direction": direction,
#         "contact_intensity": contact_intensity,
#         "prct_language_used": prct_language_used,
#     }


def decode_edge(code: str, node_a: int, node_b: int) -> dict:

    source_node = None
    target_node = None
    mutual_connection = False
    contact_intensity = None
    prct_language_used = None

    try:
        if code[0] == "1":  # out edge
            source_node = node_a
            target_node = node_b
        elif code[0] == "2": # in edge
            source_node = node_b
            target_node = node_a
        elif code[0] == "3":
            mutual_connection = True
            print("mutual connection")
        try:
            contact_intensity = int(code[0]) / 5  # normalised
        except:
            contact_intensity = 0.
        try:
            prct_language_used = int(code[2]) / 9  # normalised
        except:
            prct_language_used = 0.
    except:
        print(code)

    
    if mutual_connection:
        return [
            {
                "source": node_a,
                "target": node_b,
                "contact_intensity": contact_intensity,
                "prct_language_used": prct_language_used,
            },
            {
                "source": node_b,
                "target": node_a,
                "contact_intensity": contact_intensity,
                "prct_language_used": prct_language_used,
            }
        ]
    
    return [
        {
            "source": source_node,
            "target": target_node,
            "contact_intensity": contact_intensity,
            "prct_language_used": prct_language_used,
        }
    ]

def get_node(col_name: str) -> str:
    return col_name.split("@")[1]
        

In [196]:
data_table = []

for _, row in ddf.iterrows():
    source_node = row["metric_id"]
    for col_node, code in row[:-1].items():
        referred_node = get_node(col_node)
        cc = decode_edge(code, source_node, referred_node)
        data_table.extend(cc)

mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual connection
mutual con

In [199]:
edge_list = pd.DataFrame(data_table).dropna(axis=0)
edge_list

,source,target,contact_intensity,prct_language_used
1,2,1,0.4,0.0
2,1,3,0.2,0.0
5,1,6,0.2,0.0
6,1,7,0.2,0.0
7,1,8,0.2,0.0
...,...,...,...,...
1799,34,41,0.4,0.0
1802,37,41,0.4,0.0
1803,38,41,0.4,0.0
1804,39,41,0.4,0.0


In [205]:
# edge_list["prct_language_used"].unique()
# edge_list["contact_intensity"].unique()
# edge_list["target"].unique()
edge_list["source"].unique()

array(['2', 1, '10', '11', '14', '15', '20', '21', '24', '26', '30', '31',
       '35', '40', 2, '4', '5', '12', '13', '19', '23', '25', '27', '33',
       '36', '37', '39', 3, '6', '17', '32', 4, '18', 5, '9', '22', '29',
       '34', '38', '41', 6, '28', '1', '3', 7, 8, '16', 9, 10, '8', 12,
       13, 14, 15, '7', 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], dtype=object)

In [206]:
edge_list.to_csv("first_shot_edge_list.csv")